# Import Libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# Load and preprocess data

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255
Y_train = to_categorical(y_train, num_classes=10)
Y_test = to_categorical(y_test, num_classes=10)


# CNN model & Layers

In [ ]:
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((1, 1)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, Y_train, batch_size=128, epochs=15, validation_data=(X_test, Y_test))


C:\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 30s 57ms/step - accuracy: 0.8328 - loss: 0.5455 - val_accuracy: 0.9680 - val_loss: 0.1075
Epoch 2/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 27s 56ms/step - accuracy: 0.9550 - loss: 0.1500 - val_accuracy: 0.9785 - val_loss: 0.0708
Epoch 3/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 26s 55ms/step - accuracy: 0.9690 - loss: 0.1021 - val_accuracy: 0.9808 - val_loss: 0.0589
Epoch 4/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 27s 58ms/step - accuracy: 0.9760 - loss: 0.0780 - val_accuracy: 0.9826 - val_loss: 0.0526
Epoch 5/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 27s 58ms/step - accuracy: 0.9796 - loss: 0.0657 - val_accuracy: 0.9833 - val_loss: 0.0527
Epoch 6/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 27s 58ms/step - accuracy: 0.9827 - loss: 0.0550 - val_accuracy: 0.9852 - val_loss: 0.0452
Epoch 7/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 27s 58ms/step - accuracy: 0.9847 - loss: 0.0479 - val_accuracy: 0.9839 - val_loss: 0.0477
Epoch 8/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 27s 58ms/step - accuracy: 0.9873 - loss: 0.0392 - 

# Evaluate the model & Predict

In [ ]:
test_scores = model.evaluate(X_test, Y_test, verbose=2)
print(f"Test Loss: {test_scores[0]:.4f}")
print(f"Test Accuracy: {test_scores[1] * 100:.2f}%")

def predict_with_confidence(model, images):
    predictions = model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    confidence_levels = np.max(predictions, axis=1)
    return predicted_classes, confidence_levels

sample_images = X_test[:10]
true_labels = y_test[:10]
predicted_classes, confidence_levels = predict_with_confidence(model, sample_images)

for i, (true_label, predicted_class, confidence) in enumerate(zip(true_labels, predicted_classes, confidence_levels)):
    print(f"Image {i + 1}: True Label = {true_label}, Predicted = {predicted_class}, Confidence = {confidence:.2f}")


# Confusion Matrix

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=1)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(10))
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()


# Training and Validation Accuracy Graphs

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


# Training and Validation Loss Graphs

In [ ]:

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


# Saving Model

In [ ]:
model.save('mnist_cnn_model.h5')